In [37]:
#BP神经网络
%matplotlib qt5
import os
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import Callback
from sklearn.preprocessing import scale
import matplotlib.pyplot as plt  
from mpl_toolkits.mplot3d import Axes3D  

In [38]:
#训练数据
X = []
Y = []
Z = []
#path = '/Users/liuyingjie/Downloads/data/multi/1/'
path = '/Users/liuyingjie/Downloads/data/two/6/'
filenames = os.listdir(path)
indexs = []
for filename in filenames:
    if filename.startswith("."):
        continue
    indexs.append(int(filename))
indexs.sort()
#print(indexs)
for index in indexs:
    file_path = path + str(index)
    with open(file_path,'r',encoding='utf8') as f:
        lines=f.readlines()
    for num in range(102, 122):
        #print(lines[num])
        temps = lines[num].split()
        X.append(float(temps[2])/100)
        Y.append(float(temps[3])/100)
    for line in lines:
        temps = line.split()
        size = len(temps)
        if(size == 11 and temps[0]=='10000000'):
            Z.append(int(float(temps[1])*37000000000))
            Z.append(int(float(temps[6])*37000000000))
        if(size == 16 and temps[0] == '10000000'):
            Z.append(int(float(temps[1])*37000000000))
            Z.append(int(float(temps[6])*37000000000))
            Z.append(int(float(temps[11])*37000000000))
            
print(Z)

[3404, 3887, 4368, 4792, 5076, 5123, 4979, 4625, 4196, 3752, 3344, 2989, 2666, 2403, 2169, 1976, 1820, 1651, 1517, 1404, 1289, 1385, 1515, 1606, 1734, 1869, 2039, 2215, 2397, 2625, 2878, 3108, 3342, 3544, 3688, 3739, 3676, 3517, 3264, 2981, 2673, 3189, 3843, 4657, 5648, 6736, 7787, 8456, 8494, 7910, 6947, 5940, 5020, 4267, 3684, 3203, 2819, 2516, 2257, 2052, 1864, 2032, 2219, 2437, 2687, 2988, 3347, 3807, 4410, 5157, 6110, 7227, 8350, 9151, 9278, 8631, 7461, 6199, 5069, 4129]


In [39]:
#预测数据
Point1=[90,110]#预测场左上角坐标
Point2=[110,90]#预测场右下角坐标
X_interval=0.3
Y_interval=-0.3
X_p = []
Y_p = []
X_start=Point1[0]-X_interval
Y_start=Point1[1]-Y_interval
while abs(Point2[0] -X_start)>=abs(X_interval)or abs(Point2[1]-Y_start)>=abs(Y_interval):
    X_start=Point1[0]-X_interval
    Y_start+=Y_interval
    while abs(Point2[0] -X_start)>=abs(X_interval):
        X_start+=X_interval
        X_p.append(X_start)
        Y_p.append(Y_start)
#print(X_p)
#print(Y_p)
#print(len(X_p))
#print(len(Y_p))

In [40]:
X_pred = scale(np.array(X_p))
Y_pred = scale(np.array(Y_p))
X_pred_ = np.array([X_pred,Y_pred])
X = scale(np.array(X))
Y = scale(np.array(Y))
X_ = np.array([X,Y])
Y_ = scale(np.array(Z))

In [53]:
from sklearn.svm import SVR
#注意，C的值很关键，没有1000不出三个点
clf = SVR(C=100, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
    gamma='auto_deprecated', kernel='rbf', max_iter=-1, shrinking=True,
    tol=0.001, verbose=False)

clf.fit(X_.T, Y_.reshape((-1,1))) 
Y_pred = clf.predict(X_pred_.T)

ans=[]
predict=[]
xLength=int(abs(Point2[0]-Point1[0])/abs(X_interval))
yLength=int(abs(Point2[1]-Point1[1])/abs(Y_interval))
i=0
while i <len(X_p):
    j=0
    res=[]
    while j<=xLength:
        r=[X_p[i],Y_p[i],Y_pred[i]]
        res.append(r)
        i+=1
        j+=1
    ans.append(res)
#print(ans) 
res=[]
i=1
while i<yLength:
    j=1
    while j<xLength:
        if ans[i][j][2]>ans[i-1][j][2] and ans[i][j][2]>ans[i+1][j][2] and ans[i][j][2]>ans[i][j-1][2] \
           and ans[i][j][2]>ans[i][j+1][2] and ans[i][j][2]>ans[i-1][j-1][2] and ans[i][j][2]>ans[i-1][j+1][2] \
            and ans[i][j][2]>ans[i+1][j-1][2] and ans[i][j][2]>ans[i+1][j+1][2]:
            res.append(ans[i][j])
        j+=1
    i+=1
res.sort(key=lambda k:k[2],reverse = True)
print(res[:4])

[[95.09999999999995, 97.40000000000012, 2.60603425680862], [101.69999999999989, 105.50000000000004, 2.0489200723332877]]


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [54]:
ax = plt.figure().add_subplot(111, projection = '3d')  
ax.scatter(X_pred_[0], X_pred_[1], Y_pred, c = 'b', marker = 'o') #点为红色三角形  
ax.set_xlabel('X Label')  
ax.set_ylabel('Y Label')  
ax.set_zlabel('Z Label')  
plt.show()

In [20]:
ax = plt.figure().add_subplot(111, projection = '3d')  
ax.scatter(X_[0], X_[1], Y_, c = 'b', marker = 'o') #点为红色三角形  
ax.set_xlabel('X Label')  
ax.set_ylabel('Y Label')  
ax.set_zlabel('Z Label')  
plt.show()